In [1]:
!pip install detoxify

### Classifier

In [2]:
from transformers import CamembertForSequenceClassification, CamembertTokenizer
import torch

# ✅ Load model & tokenizer from Hugging Face
repo_name = "Minervus00/camembert-news-classifier"
model = CamembertForSequenceClassification.from_pretrained(repo_name)
tokenizer = CamembertTokenizer.from_pretrained(repo_name)
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/374 [00:00<?, ?B/s]

In [3]:
label_map = {0: 'Autres', 1: 'Culture', 2: 'Politique', 3: 'Santé', 4: 'Sport', 5: 'Sécurité', 6: 'Économie'}
label_map[0]

'Autres'

In [4]:
def classify(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs={k:v.to(model.device) for k,v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class_id = outputs.logits.argmax(dim=-1).item()
    return label_map[predicted_class_id]

In [ ]:
content = """
𝐂𝐨𝐮𝐩𝐞 𝐝𝐮 𝐦𝐨𝐧𝐝𝐞 𝐔𝟏𝟕 𝐐𝐚𝐭𝐚𝐫 : 𝐥𝐞𝐬 𝐞́𝐭𝐚𝐥𝐨𝐧𝐬 𝐜𝐫𝐞́𝐞𝐧𝐭 𝐥’𝐞𝐱𝐩𝐥𝐨𝐢𝐭 𝐜𝐨𝐧𝐭𝐫𝐞 𝐥𝐚 𝐌𝐚𝐧𝐧𝐬𝐜𝐡𝐚𝐟𝐭 𝐞𝐭 𝐟𝐢𝐥𝐞𝐧𝐭 𝐞𝐧 𝐡𝐮𝐢𝐭𝐢𝐞̀𝐦𝐞𝐬
Les Étalons U17 ont créé le samedi 15 novembre 2025, la sensation en s’imposant 1-0 face à la Mannschaft d’Allemagne, décrochant ainsi leur billet pour les huitièmes de finale de la coupe du monde U17.
Les Étalons devront désormais se tourner vers leur prochain défi : l’Ouganda, tombeur un peu plus tôt dans l’après-midi du Sénégal, également sur le score de 1-0. La rencontre est programmée pour le mardi 18 novembre 2025.
DCRP/MSJE
"""

# content = """
# 5e édition de la Coupe des ambassades : Le Canada étrille l’Algérie dans le match d’ouverture (8-0)
# La 5e édition de la Coupe des ambassades en football a été lancée dans l’après-midi de ce samedi 15 novembre 2025 à Ouagadougou, par la secrétaire générale du ministère des Sports, Colette Ouédraogo, représentant le ministre Roland Somda. En match d’ouverture, le Canada a été sans pitié face à l’Algérie, étrillée 8-0. Dix-sept équipes (ambassades accréditées au Burkina Faso) réparties en quatre poules prennent part à cette édition 2025 de la Coupe des ambassades.

# """

# content = """

# """

print(classify(content))       # → 'Sport'

Sport


### Toxicity and sensibility

In [5]:
from detoxify import Detoxify
detoxify_model = Detoxify('multilingual', device="cuda")

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.4-alpha/multilingual_debiased-0b549669.ckpt" to /root/.cache/torch/hub/checkpoints/multilingual_debiased-0b549669.ckpt


100%|██████████| 1.04G/1.04G [00:11<00:00, 97.7MB/s]


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [7]:
def toxicity_analysis(text):
    results = detoxify_model.predict(text)
    sensibility = (results['toxicity'] > 0.8) or (results['sexual_explicit'] > 0.8) or (results['obscene'] > 0.8)
    return float(results['toxicity']), bool(sensibility)

In [ ]:
# content = """
# Du 19 octobre au 2 novembre 2025, les Forces combattantes burkinabè ont mené une série d’opérations coordonnées qui ont permis de neutraliser de nombreux terroristes et de détruire plusieurs de leurs bases dans les régions des Koulsé, du Goulmou, du Liptako et de Bankui a appris l’AIB.

# Le 19 octobre, les unités du 8e Bataillon d’intervention rapide (BIR), appuyées par les équipes ESRI 1 et Léopard, ont affronté des terroristes à motos dans la zone de Pibaoré. Après de violents échanges de tirs, les assaillants ont été poursuivis par voie aérienne et neutralisés.

# Deux jours plus tard, le 21 octobre, une attaque contre un poste avancé à Kilma a été repoussée par les forces terrestres avant d’être suivie par des frappes sur les fuyards. Le même jour, un quartier général terroriste à Korko, au nord de Barsalogho, a été détruit.
# Le 22 octobre, plusieurs criminels ont été neutralisés dans leur base de Demniol, à l’est de Silmangué.

# Le 24 octobre, une opération aérienne d’envergure a été déclenchée après la détection d’un important convoi de terroristes au nord de Yamba. La colonne ennemie a été entièrement anéantie, avec la saisie d’un arsenal important.
# Le 25 octobre, des frappes aériennes successives ont détruit un dispositif ennemi tandis qu’une tentative d’attaque contre les positions de défense à Koumbango (province du Bam) a été vigoureusement repoussée. Les terroristes en déroute ont abandonné leurs armes, plusieurs corps et un de leurs éléments capturés.

# Dans la région de Bankui, le 29 octobre, des unités combattantes basées à Tougan, province du Sourou ont intercepté un déplacement suspect. Les vecteurs aériens sont intervenus rapidement, neutralisant les ennemis et détruisant leur matériel.
# Le 30 octobre, un groupe planifiant des attaques contre Kongoussi, Tikaré et Kossouka a été pris pour cible à Nasseré. Les frappes ont détruit leurs caches et éliminé les rescapés. Une autre frappe, menée dans la même journée, a visé des terroristes endormis, suivie d’une intervention des forces spéciales pour le ratissage.

# Le 31 octobre, un regroupement terroriste dans le village de Temnaoré, près de Kossouka, a été repéré puis détruit par des frappes coordonnées détruisant leurs équipements.
# Le 1er novembre, une unité du 19e BIR a tendu une embuscade à un groupe de guetteurs ennemis venus espionner ses positions. Tous les membres du groupe ont été neutralisés.
# Enfin, le 2 novembre, une frappe chirurgicale menée au nord de Barsalogho a visé une réunion de hauts cadres terroristes, neutralisant plusieurs chefs et détruisant leurs véhicules et dépôts d’armes.

# Ces différentes opérations menées avec rigueur et coordination traduisent la volonté ferme des Forces combattantes de libérer totalement le territoire national de l’emprise des terroristes.
# Elles illustrent également la montée en puissance du dispositif de défense, marquée par la complémentarité entre forces terrestres, unités spéciales et moyens aériens, désormais capables d’intervenir simultanément sur plusieurs fronts.
# Selon les autorités militaires, les opérations se poursuivent sans relâche, avec pour objectif la restauration de la libre circulation des personnes et des biens, et la reconquête totale du territoire national.
# """

content = """
Mr le sous Lieutenant Sedogo, avec tout le respect que je vous dois, je penses que etant au front, vous devez etre différent de ceux qui dorment dans les foutus rond-points! Nous avons recu votre message que vous avez passé à L'église protestante,  Catholique et à la mosquée à Boulsa.
" Ko il faut que les gens evitent de partager les videos avec les ennemis tels que Conombo Alassane, Naïm Touré, Oumar Coulibaly et autres.."   ce n'est par parce qu'on parle que le pays est attaqué, c'est parce que le pays est attaqué qu'on parle.
Est-ce-que c'est  nous qui attaquons le Burkina Faso? Depuis que je suis sur la toile j'évite de m'adresser aux chefs de détachements, aux  FDS et VDP engagés au front, mon combat s'adresse a celui qui a fait le coup D'état et a promis de libérer le pays en 6 mois. Donc si vous avez échoué a protéger Boulsa/ Namentenga, ce n'est pas la faute à Naim Touré, Oumar Coulibaly, Alassane Conombo officiel et autres dont tu accuse fierement devant les civils dans les lieux de cultes. .
"""

# content = """
# C'est la désillusion totale dans les rangs des propagandistes. Ci-dessous des images de ce qu'on pourrait qualifier de "pistes rurales" de la ZONE INDUSTRIELLE DE LA CAPITALE ÉCONOMIQUE/BOBO. Images bien-sûr postées par une page de propagande du régime putschiste d'ibrahim traoré.
# Pourtant #Faso_Mêbo (l'une des plus grosses arnaques des bidasses) était censé avoir réglé tous les problèmes de voirie au bled 🇧🇫
#  Djaa zéro lembourdji ! Regardez ce sont eux-mêmes qui disent que ça ne va pas 😅
# Lorsqu'on vous dit que vous n'êtes qu'un ramassis de laga laga tô (les pro ib), vous vous fâchez... pourtant les faits sont têtus : zéro amélioration de vos conditions de vie. Zéro évolution. Zéro souveraineté. #Faso_Mêbo est devenu #Faso_Gnèdbô

# """

# content = """
# Cest ce mardi 29 Avril 2025 à 14h au camp Sangoulé Lamizana que sera enterré Le sous lieutenant Natama Yacouba ( 26 ans) du BIR 16 tombé le 23 Avril 2025 à Kongoussi.
# NB: Je répète IB et leurs amis n'ont pas de frère ni de sœurs au front. Ce sont les enfants des autres qui sont envoyés a l'abattoir pour leurs permettre dans leurs climatisations de parler de révolution progressiste
# """

toxicity_analysis(content)

(0.26884058117866516, True)

### Analysis

In [ ]:
cmts = {"comments": [{"date": "24 janvier 2024 21:10", "text": "Une partie du Burkina a choisi son camp et l’imposé au Peuple. il y a des pays ou un citoyen a un droit et des pays ou le pouvoir a le droit absolu. Il n’y a pas de bon ni de mauvais. l’histoire nous dira !!!", "replies": []}, {"date": "25 janvier 2024 03:30", "text": "J’ai souvent relevé le fait qu’il n’a jamais été fait un bilan froid et depassionné de la période SANKARA, qui a depuis fait Héros de la Nation. Sous SANKARA, nous étions dans une logique similaire : Celui qui n’est pas avec nous est contre nous ! Nous savons tous ou cela a conduit le pays. Aussi longue est la nuit, le jour finit par se lever ! Nous attendrons patiemment l’issue de cette aventure, vu qu’il est inutile, voire contre-productif d’élever la moindre voix dissonante. Ce n’est pas la peur loin s’en faut. C’est le réalisme. Il n’est pas utile de grossir le nombre des \"enlevés\". Quand ces gens arriverons au terme de leur logique, ils comprendront sûrement. Ce qui se passe est la preuve même que le coup d’état est la pire forme d’accession au pouvoir, et que la démocratie, en dépit de toutes ses insuffisances, est la moins mauvaise des formes de gouvernement. Et que contrairement à une idée que l’on nous serine à tout bout de champ, il n’y a pas de démocratie à l’africaine, de démocratie tropicalisée, et démocratie à l’occidentale. Il y a la DÉMOCRATIE. Bien entendu elle est une quête permanente, elle n’est jamais définitivement acquise. Quand on constate que les États Unis pourraient voir le retour au pouvoir de Donald TRUMP qui est tout sauf un démocrate, on comprend que la quête de la démocratie est un combat de tous les jours, et qu’il n’y a pas de trêve possible. Quand nos gens comprendront que ce qu’ils font peut se retourner contre eux, peut-être changeront-ils. En attendant on ne peut que prier pour le maintien de l’intégrité physique de OUEDRAOGO Ablassé, Sansan Anselme KAMBOU, Evrard SOMDA, et tous les autres. Que Dieu aide le BURKINA !", "replies": [{"date": "25 janvier 2024 10:55", "text": "@Alpha2025 : On a dejà fait ce bilan froid et depassionné et c’est le resultat de ce bilan (actions positives -actions negatives) qui fait que Sankara est Héros de la nation. Si tu veux tu peux à ton tour faire seul ton bilan froid et depassionné et dire ce que tu veux. Personne n’ a dit que sous Sankara toutes les actions étaient positives, c’est le bilan qui est largement positif. Par exemple la France que tu prend comme exemple, sous la revolution francaisse il ya eue beaucoup d’atrocités et de victimes mais les francais continuent de celebrer la revolution francaise chaque année. Sous le General Degaulle qui est un Héro de la nation francaise, il ya eue de nombreux francais qui on été accusés de collabo (collaborateurs des Nazis) et qui ont été executés, mais aucun francais ne remet en cause le statut de Héros de la nation de Degaullle. Ou bien tu te crois plus intelligents que tous les francais ?"}, {"date": "25 janvier 2024 11:08", "text": "Tu as tout dit juste mon frere... Dieu exausse nos prieres.A bon entendeur, Salut."}, {"date": "25 janvier 2024 13:55", "text": "Vivement ce bilan de la révolution pour servir pour la postérité et éviter ainsi les répétitions inutiles qui coûtent parfois chères à l’Etat. Qui a pensé aux enfants qui ont définitivement abandonné les bancs du fait du licenciement de leurs parents enseignants ? Et ceux qui en sont morts ou devenus fous ? Et la baisse du niveau de l’enseignement ? Cela n’est que la partie visible de l’iceberg Merci à vous Alpha2025 pour cette belle analyse."}]}, {"date": "25 janvier 2024 09:01", "text": "Voila pourquoi nos magistrats et personnels de la justice se doivent d’être fermes et de ne pas violer eux mêmes les règles et principes de droit pour plaire au Prince et potentat du moment.", "replies": []}, {"date": "25 janvier 2024 09:04", "text": "Sous nos tropiques !!! Et c’est pas la faute de la France !!! \"Qui a la force !!!! A le droit.\"", "replies": []}, {"date": "25 janvier 2024 10:05", "text": "Bonjour, Belle analyse. Quelque soit ta position et ton pouvoir, il faut être juste et humble car l’humilité précède la gloire et l’orgueil précède la chute dit la Bible.", "replies": []}, {"date": "25 janvier 2024 10:36", "text": "On a demandé aux militaires français de partir parce qu’on voulait notre souveraineté. Pourtant, il y a 300 militaires russes qui s’apprêtent à débarquer (source : Grey Zone, Telegram) : – la formation des militaires burkinabè, – la sécurité du président, – le soutien des patrouilles. La gouvernance au Burkina manque de vision, de constance. Seul, l’intérêt du moment prime.", "replies": []}, {"date": "25 janvier 2024 11:05", "text": "Il faut qu’on cherche à savoir si la loi au Burkina autorise l’Etat a procédé de la sorte pour interpeller ses citoyens. Moi personnellement, je ne le sais pas. Cependant je sais que sous la présidence de Roch, on voyait des forces armées cagoulées pour mener certaines opérations (attaque terro de Ragnongo). Donc s’il n’existe pas de disposition légale, les citoyens peuvent se constituer pour attaquer la pratique devant les tribunaux. Mais si la disposition existe, le débat est clos. Enfin on voit souvent en France des policiers cagoulés pour mener certaines opérations anti-terro* ou même pendant de simples manifestations (manifestations des gilets jaunes).", "replies": []}, {"date": "25 janvier 2024 13:11", "text": "De plus, Ibrahim Traore et son regime MPSR 2 doit savoir que dans l’histoire politique de l’Afrique et du Monde, des Dictatures Neocoloniales et autres ont ete Destituees pour liberer des Peuples Consernes. Ensuite, Tout Pouvoir Politique et Patriotique consequent doit servir des Interets Fondamentaux des Peuples et non pas des Individus ; mais l’on ne peut comprendre que lorsqu’on l’aura perdu. Car personne ne detient le titre foncier du Burkina Faso et nullement personne n’est nee pour commander les autres et donc l’on est la par accidents de l’histoire politique et l’on doit pas se croire plus fort, plus Intelligent ou plus ruse que les autres. Il faut eviter d’etre un Fou du Pouvoir car cela vous entraine a devenir Dictateur, Sanguinaire et parfois a la recherche effrenee des moyens Financiers et monetaire pour servir votre cause personnelle et de la posture : \"Je suis pas interesse par le Pouvoir Politique, mais c’est le Peuple qui me retient pour que je finisse les Chantiers Publics que j’ai commences\". Erreur Monumentale car nul n’est Indispensable car les Cimetieres sont bourrer des Gens Indispensables, c’est cela la Verite Incontournable. Alors il faut etre Juste et Humble dans sa Mission Principale de Securisation du Territoire National Burkinabe et que les Millions de Deplaces Internes retrouvent leurs Localites D’origines et reinstaurer la Democratie Populaire aux services Exclusifs des Citoyens ou du Peuple Burkinabe, toute autre chose ne conduira qu’a la Dictature Neocoloniale et dont la Fin devient Logique et Implacable. C’est le Sens de l’histoire. Salut", "replies": [{"date": "25 janvier 2024 20:57", "text": "Mr Ouedraogo je m incline devant la qualité de vos conseils et votre courage. Oui du courage il en faut pour dire de nos jours cette vérité de la palissade. Vous auriez pu être un simple Laudateurs et vous seriez nommé. Et cela me fait penser à un \" communicateur sur bf1 qui a été promu. J ai toujours dit que je préfère un ami qui me dit la vérité que je ne veux pas entendre qu un ami toujours intéressé. Toute personne doit en principe souffrir des actes d injustice sur tout citoyen qui qu il soit. Malheureusement au nom du ventre et du fric certains dans toute leur vie n ont jamais connus cette valeur. A qui la faute. De même que celui qui n a pas de voiture ne construit pas de garage celui qui n a jamais connu l huamanisme ne saurait accepter les droits de l’homme."}]}, {"date": "26 janvier 2024 13:34", "text": "Messieurs !!!!!!!!!!!!!!! @Koussoubé @Mr Ouedraogo Grandement plaisir et raffinement a vous lire ! Citoyens", "replies": []}]}
cmts

{'comments': [{'date': '24 janvier 2024\xa021:10',
   'text': 'Une partie du Burkina a choisi son camp et l’imposé au Peuple. il y a des pays ou un citoyen a un droit et des pays ou le pouvoir a le droit absolu. Il n’y a pas de bon ni de mauvais. l’histoire nous dira !!!',
   'replies': []},
  {'date': '25 janvier 2024\xa003:30',
   'text': 'J’ai souvent relevé le fait qu’il n’a jamais été fait un bilan froid et depassionné de la période SANKARA, qui a depuis fait Héros de la Nation. Sous SANKARA, nous étions dans une logique similaire : Celui qui n’est pas avec nous est contre nous ! Nous savons tous ou cela a conduit le pays. Aussi longue est la nuit, le jour finit par se lever ! Nous attendrons patiemment l’issue de cette aventure, vu qu’il est inutile, voire contre-productif d’élever la moindre voix dissonante. Ce n’est pas la peur loin s’en faut. C’est le réalisme. Il n’est pas utile de grossir le nombre des "enlevés". Quand ces gens arriverons au terme de leur logique, ils compre

In [17]:
def analyse_comments(comments, toxiques = None):
  if toxiques is None:
        toxiques = []

  for comment in comments:
    stuff = {}

    toxicity, sensibility = toxicity_analysis(comment["text"])
    if sensibility:
      stuff["text"] = comment["text"][:500] + "..."
      stuff["comment_sensible"] = sensibility
      stuff["toxicite_score"] = toxicity
      toxiques.append(stuff)

    if "replies" in comment.keys():
      analyse_comments(comment["replies"], toxiques)

  return toxiques

In [ ]:
analyse_comments(cmts["comments"])

[]

In [ ]:
def test_fct(a, lt=[]):
  if a == 7:
    return lt
  if a % 2 == 0:
    lt.append(a)
  a += 1
  return test_fct(a, lt)

test_fct(0)

[0, 2, 4, 6]

In [12]:
import json

with open("database.json", "r") as f:
    analysis_dic = json.load(f)

analysis_dic

{'articles': [], 'medias': []}

In [13]:
import json
from tqdm import tqdm

def open_json(file_name):
  with open(file_name) as f:
    d = json.load(f)
    return d

def analyse_media_articles(file_name):
  # open article json file
  articles = open_json(file_name)
  for article in tqdm(articles):
    buff = {}
    buff["id"] = article["id"]
    buff["media"] = article["media"]
    buff["titre"] = article["titre"]
    buff["date"] = article["date"]
    buff["url"] = article["url"]
    buff["contenu"] = article["contenu"][:500] + "..."
    buff["categorie"] = classify(article["contenu"])

    # Engagement
    engmt = article["engagement"]
    buff["engagement"] = {"likes": engmt["likes"],
                          "partages": engmt["partages"],
                          "commentaires": engmt["commentaires"] + engmt["replies"]
                          }

    # Toxicity and sensibility
    toxicity, sensibility = toxicity_analysis(article["contenu"])
    buff["sensible"] = sensibility
    buff["toxicite_score"] = toxicity

    # Commentaires toxiques
    buff["comments_sensibles"] = analyse_comments(article["comments"])

    # Add analysis to dict
    analysis_dic["articles"].append(buff)


In [14]:
presse_files = ['fasopresse_articles.json',
 'aib_articles.json',
 'lobservateur_articles.json',
 'burkina24_articles.json',
 'sidwaya_articles.json',
 'lefaso_articles.json']

In [19]:
for presse in presse_files:
  print(f"======= {presse} ======")
  analyse_media_articles(presse)

======= fasopresse_articles.json ======


100%|██████████| 50/50 [00:03<00:00, 15.30it/s]


======= aib_articles.json ======


100%|██████████| 795/795 [00:53<00:00, 14.99it/s]


======= lobservateur_articles.json ======


...

100%|██████████| 39/39 [00:01<00:00, 20.66it/s]


======= sidwaya_articles.json ======


100%|██████████| 1483/1483 [01:10<00:00, 21.12it/s]


======= lefaso_articles.json ======


100%|██████████| 2800/2800 [04:32<00:00, 10.28it/s]


In [18]:
# analysis_dic = {'articles': [], 'medias': []}

In [24]:
import json

with open("database.json", "w") as f:
    json.dump(analysis_dic, f, indent=2, ensure_ascii=False)

In [20]:
analysis_dic["articles"][0]

{'id': '6a4cb3874fb1e98e0c0849c8c456ce33',
 'media': 'FasoPresse',
 'titre': 'Opération caisses vides : Paul Kaba Thieba s’en prend au SYNTSHA',
 'date': '2018-06-12 09:42:00',
 'url': 'https://www.fasopresse.net/politique/6143-operation-caisses-vides-paul-kaba-thieba-sen-prend-au-syntsha',
 'contenu': 'Le Premier ministre Paul Kaba Thieba a qualifié, ce lundi 11 juin 2018, « d’infondée et d’illégale » l’opération caisses vides, lancée par le Syndicat des travailleurs de la santé humaine et animale (SYNTSHA).\n\nDu lundi 11 juin au lundi 18 juin 2018, les agents de santé assureront les prestations médicales sans encaisser la contrepartie financière.\n\nEt ce, pour répondre à l’appel de leur syndicat, le SYNTSHA. Cette nouvelle forme de lutte n’est pas du goût du gouvernement burkinabé qui « ne la c...',
 'categorie': 'Santé',
 'engagement': {'likes': 0, 'partages': 0, 'commentaires': 0},
 'sensible': False,
 'toxicite_score': 0.000684499682392925,
 'comments_sensibles': []}

### Media Part

In [ ]:
import json

with open("database.json", "r") as f:
    analysis_dic = json.load(f)

analysis_dic

In [ ]:
# import datetime
from collections import defaultdict

# -------------------------------------------------------
# Define score influence formula
# You can change this to whatever formula you prefer
# -------------------------------------------------------
def compute_score_influence(nb_articles, engagement_total):
    if nb_articles == 0:
        return 0.0
    return 0.7 * (engagement_total / 1000) + 0.3 * nb_articles


# -------------------------------------------------------
# Compute stats for each media
# -------------------------------------------------------
def compute_media_stats(data):

    # Dictionary to accumulate results
    stats = defaultdict(lambda: {
        "nb_articles": 0,
        "engagement_total": 0,
        "score_influence": 0.0,
        "rang": None,
        # "actif_90j": False
    })

    # today = datetime.date.today()
    # delta_90 = datetime.timedelta(days=90)

    for article in data["articles"]:
        media = article["media"]

        # Update counters
        stats[media]["nb_articles"] += 1
        stats[media]["engagement_total"] += (
            article["engagement"]["likes"]
            + article["engagement"]["partages"]
            + article["engagement"]["commentaires"]
        )

        # Check if article is within last 90 days
        # try:
        #     article_date = datetime.datetime.strptime(article["date"], "%Y-%m-%d").date()
        # except:
        #     # If format is different, adapt accordingly
        #     article_date = datetime.date.fromisoformat(article["date"])

        # if today - article_date <= delta_90:
        #     stats[media]["actif_90j"] = True

    # -------------------------------------------------------
    # Compute score influence
    # -------------------------------------------------------
    for media, info in stats.items():
        info["score_influence"] = compute_score_influence(
            info["nb_articles"],
            info["engagement_total"]
        )

    # -------------------------------------------------------
    # Ranking by score_influence (descending)
    # -------------------------------------------------------
    ranked = sorted(stats.items(), key=lambda x: x[1]["score_influence"], reverse=True)

    for rank, (media, info) in enumerate(ranked, start=1):
        info["nom"] = media
        info["rang"] = rank

    # Save results back into data["medias"]
    data["medias"] = [info for media, info in ranked]

    return data


# -------------------------------------------------------
# Example usage
# -------------------------------------------------------
# data = { "articles": [...], "medias": [] }
data = compute_media_stats(analysis_dic)
# print(json.dumps(data, indent=2, ensure_ascii=False))
data["medias"]

In [ ]:
import json

with open("../data/processed/final_db1.json", "w") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

## Socials

In [3]:
import json
from datetime import datetime

# -----------------------------------------------------------
# Convert one source Facebook post to your target article dict
# -----------------------------------------------------------
def map_item(src):
    # Parse and format date
    # example input: "2025-08-01T14:06:07.000Z"
    dt = datetime.fromisoformat(src["time"].replace("Z", ""))
    formatted_date = dt.strftime("%Y-%m-%d")

    # Build target item
    item = {
        "id": src["postId"],
        "media": src["pageName"],
        "titre": src["text"][:30],
        "date": formatted_date,
        "url": src["url"],
        "contenu": src["text"],

        "engagement": {
            "commentaires": src.get("comments", 0),
            "replies": 0,
            "likes": src.get("likes", 0),
            "partages": src.get("shares", 0)
        },

        "comments": []     # requested: empty list
    }

    return item


# -----------------------------------------------------------
# Convert full JSON file
# -----------------------------------------------------------
def convert_file(src_path, dst_path):
    # Load source file
    with open(src_path, "r", encoding="utf-8") as f:
        src_data = json.load(f)

    out_items = []

    # Map each item
    for obj in src_data:
        try:
            out_items.append(map_item(obj))
        except Exception as e:
            print(f"⚠️ Error processing item {obj.get('postId')}: {e}")

    # Write new JSON list
    with open(dst_path, "w", encoding="utf-8") as f:
        json.dump(out_items, f, indent=2, ensure_ascii=False)

    print(f"✔ Converted {len(out_items)} items → {dst_path}")


# -----------------------------------------------------------
# Example usage
# -----------------------------------------------------------
# convert_file("source.json", "converted.json")

In [4]:
pages = [
    ("../../scrapers/Facebook scrapers/BF1TV_posts.json", "../raw/BF1TV_articles.json"),
    ("../../scrapers/Facebook scrapers/NaimToure_posts.json", "../raw/NaimToure_articles.json"),
    ("../../scrapers/Facebook scrapers/OmegaFM_posts.json", "../raw/OmegaFM_articles.json"),
    ("../../scrapers/Facebook scrapers/RTB_posts.json", "../raw/RTB_articles.json")
]
for src, tgt in pages:
    convert_file(src, tgt)

✔ Converted 200 items → ../raw/BF1TV_articles.json
✔ Converted 200 items → ../raw/NaimToure_articles.json
✔ Converted 200 items → ../raw/OmegaFM_articles.json
✔ Converted 200 items → ../raw/RTB_articles.json


### Merge

In [ ]:
import json

with open("database2.json", "r") as f:
    analysis_dic = json.load(f)

with open("final_db1.json", "r") as f:
    final_dic = json.load(f)


In [9]:
len(final_dic["articles"])

5533

In [10]:
final_dic["articles"].extend(analysis_dic["articles"])

In [11]:
len(final_dic["articles"])

6333

In [12]:
# import datetime
from collections import defaultdict

# -------------------------------------------------------
# Define score influence formula
# You can change this to whatever formula you prefer
# -------------------------------------------------------
def compute_score_influence(nb_articles, engagement_total):
    if nb_articles == 0:
        return 0.0
    return 0.7 * (engagement_total / 1000) + 0.3 * nb_articles


# -------------------------------------------------------
# Compute stats for each media
# -------------------------------------------------------
def compute_media_stats(data):

    # Dictionary to accumulate results
    stats = defaultdict(lambda: {
        "nb_articles": 0,
        "engagement_total": 0,
        "score_influence": 0.0,
        "rang": None,
        # "actif_90j": False
    })

    # today = datetime.date.today()
    # delta_90 = datetime.timedelta(days=90)

    for article in data["articles"]:
        media = article["media"]

        # Update counters
        stats[media]["nb_articles"] += 1
        stats[media]["engagement_total"] += (
            article["engagement"]["likes"]
            + article["engagement"]["partages"]
            + article["engagement"]["commentaires"]
        )

        # Check if article is within last 90 days
        # try:
        #     article_date = datetime.datetime.strptime(article["date"], "%Y-%m-%d").date()
        # except:
        #     # If format is different, adapt accordingly
        #     article_date = datetime.date.fromisoformat(article["date"])

        # if today - article_date <= delta_90:
        #     stats[media]["actif_90j"] = True

    # -------------------------------------------------------
    # Compute score influence
    # -------------------------------------------------------
    for media, info in stats.items():
        info["score_influence"] = compute_score_influence(
            info["nb_articles"],
            info["engagement_total"]
        )

    # -------------------------------------------------------
    # Ranking by score_influence (descending)
    # -------------------------------------------------------
    ranked = sorted(stats.items(), key=lambda x: x[1]["score_influence"], reverse=True)

    for rank, (media, info) in enumerate(ranked, start=1):
        info["nom"] = media
        info["rang"] = rank

    # Save results back into data["medias"]
    data["medias"] = [info for media, info in ranked]

    return data


# -------------------------------------------------------
# Example usage
# -------------------------------------------------------
# data = { "articles": [...], "medias": [] }
data = compute_media_stats(final_dic)
# print(json.dumps(data, indent=2, ensure_ascii=False))
data["medias"]

[{'nb_articles': 2800,
  'engagement_total': 10589,
  'score_influence': 847.4123,
  'rang': 1,
  'nom': 'Lefaso.net'},
 {'nb_articles': 1483,
  'engagement_total': 0,
  'score_influence': 444.9,
  'rang': 2,
  'nom': 'Sidwaya'},
 {'nb_articles': 200,
  'engagement_total': 277304,
  'score_influence': 254.11279999999996,
  'rang': 3,
  'nom': 'rtburkina'},
 {'nb_articles': 200,
  'engagement_total': 271545,
  'score_influence': 250.0815,
  'rang': 4,
  'nom': 'RadioOmegafmOfficiel'},
 {'nb_articles': 795,
  'engagement_total': 0,
  'score_influence': 238.5,
  'rang': 5,
  'nom': 'AIB'},
 {'nb_articles': 200,
  'engagement_total': 224631,
  'score_influence': 217.24169999999998,
  'rang': 6,
  'nom': 'BF1TV'},
 {'nb_articles': 366,
  'engagement_total': 0,
  'score_influence': 109.8,
  'rang': 7,
  'nom': "L'Observateur Paalga"},
 {'nb_articles': 200,
  'engagement_total': 36103,
  'score_influence': 85.2721,
  'rang': 8,
  'nom': 'naim.toure'},
 {'nb_articles': 50,
  'engagement_total'

In [14]:
import json

with open("final_db.json", "w") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)